In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',]
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6)
})

output_path = rd.rootdir/'output'/'fig_architecture'
cache_path = rd.rootdir/'output'/'fig_overview'/'data.gzip'

# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)


In [ ]:
# Create dicts to specify colors/markers
metadata['miR_'] = (metadata['miR'].isin(['none','na'])).map({True: '–', False:'+'})
metadata['ts_'] = (metadata['ts'].isin(['none','na'])).map({True: '–', False:'+'})
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color'] # colored by design
construct_markers = metadata_dict['markers']

# Create color palette for comparing designs
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='miR'), 'color'] = base.colors['purple']
metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR_loc']=='UTR'), 'color'] = metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR_loc']=='UTR'), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'color'] = base.colors['blue']
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF6x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF6x1'), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF4x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF4x1'), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==1), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==2), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['design']==3), 'color'] = base.colors['red']

metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==1), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==1), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==3), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['design']==3), 'color'].apply(base.get_dark_color)

# markers
metadata2['markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'
metadata2.loc[(metadata2['group']=='ts3'), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='ts5'), 'markers'] = 'v'
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'markers'] = 'D'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'
metadata2.loc[(metadata2['group']=='ts5'), 'linestyle'] = '--'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
designs_palette = metadata_dict2['color']
designs_markers = metadata_dict2['markers']

In [ ]:
# Gate cells
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]

In [ ]:
### Bin by marker quantiles
by = ['construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])


### Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')


### Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)


### Compute slope for all constructs
fits = stats_quantiles.groupby(['construct','exp','biorep'])[stats_quantiles.columns].apply(base.get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')

In [ ]:
metadata['ts_label'] = metadata['ts_kind'].replace({'na': 'base', 'NT': 'unreg.', 'T': 'circuit'})

Two-gene data (`data2`)

In [ ]:
exp93_path = base_path/'2024.04.14_exp93'/'export'

plates = pd.DataFrame({
    'data_path': [exp93_path/'plate1', exp93_path/'plate2', exp93_path/'plate3',],
    'yaml_path': [exp93_path/'exp93_wells.yaml']*3,
    'biorep': [1, 2, 3],
    'exp': ['exp93']*3,
})

#cache_path = rd.rootdir/'output'/'fig_architecture'/'data2.gzip'
cache_path = rd.rootdir/'output'/'two-gene'/'data.gzip'

# Load data
data2 = pd.DataFrame()
if cache_path.is_file(): data2 = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data2 = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data2 = data2[data2[c]>0]
    
    data2.dropna(inplace=True)
    data2.to_parquet(rd.outfile(cache_path))

In [ ]:
# Add metadata for constructs
data2 = data2.merge(metadata, how='left', on='construct')
metadata_construct2 = pd.read_excel(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct2-metadata.xlsx')
data2 = data2.merge(metadata_construct2, how='left', on='construct2')
data2['condition'] = data2['construct'] + '_' + data2['construct2']

# Rename far-red channel
data2.rename(columns={'SNAP-647-A': 'iRFP-A'}, inplace=True)

display(data2)

In [ ]:
metadata_comb = data2.drop_duplicates('condition')[['construct','construct2','condition']]
metadata_comb = metadata_comb.merge(metadata, how='left', on='construct')
metadata_comb = metadata_comb.merge(metadata_construct2, how='left', on='construct2')

# Create color palette
metadata_comb.loc[metadata_comb['design']==1, 'color'] = base.colors['teal']
metadata_comb.loc[metadata_comb['design']==2, 'color'] = base.colors['orange']
metadata_comb.loc[metadata_comb['design']==3, 'color'] = base.colors['red']
metadata_comb.loc[metadata_comb['gene']=='2T', 'color'] = base.colors['green']
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='EF1a'), 'color'] = base.colors['purple']
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='U6'), 'color'] = base.colors['blue']
metadata_comb.loc[(metadata_comb['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata_comb.loc[((metadata_comb['design']==1) & (metadata_comb['ts_kind']=='NT')), 'color'] = metadata_comb.loc[((metadata_comb['design']==1) & (metadata_comb['ts_kind']=='NT')), 'color'].apply(base.get_light_color)
metadata_comb.loc[((metadata_comb['design']==3) & (metadata_comb['ts_kind']=='NT')), 'color'] = metadata_comb.loc[((metadata_comb['design']==3) & (metadata_comb['ts_kind']=='NT')), 'color'].apply(base.get_dark_color)
metadata_comb.loc[metadata_comb['group'].isin(['base','marker']), 'color'] = 'black'

# markers
metadata_comb['markers'] = 'X'
metadata_comb.loc[metadata_comb['gene']=='1T', 'markers'] = 'o'
metadata_comb.loc[metadata_comb['gene']=='2T', 'markers'] = 'D'
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='EF1a'), 'markers'] = '^'
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='U6'), 'markers'] = 'v'

metadata_comb['condition'] = metadata_comb['construct'] + '_' + metadata_comb['construct2']
metadata_comb['ts_label'] = metadata_comb['ts_kind'].replace({'na': 'base', 'NT': 'unreg.', 'T': 'circuit'})
display(metadata_comb)

metadata_dict = metadata_comb.set_index('condition').to_dict('dict')
two_gene_palette = metadata_dict['color']
two_gene_markers = metadata_dict['markers']

In [ ]:
metadata_comb.loc[(metadata_comb['ts_loc']=='5\''), 'color'] = metadata_comb.loc[(metadata_comb['ts_loc']=='5\''), 'color'].apply(base.get_light_color)
metadata_dict = metadata_comb.set_index('condition').to_dict('dict')
two_gene_palette2 = metadata_dict['color']

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data2[data2['construct']=='GEEC555'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Indicate which channels are relevant for each experiment
gates['marker'] = 'iRFP-A'
gates['output'] = 'mRuby2-A'

# Add missing iRFP gate
gate_iRFP = 2.5e2
gates['iRFP-A'] = [gate_iRFP]
display(gates)

# Gate data
data2 = data2.groupby('exp')[data2.columns].apply(lambda x: base.gate_data(x,gates))
data2.reset_index(inplace=True, drop=True)
max = 1e6
df = data2[(data2['expressing']) & (data2['output']<max)]

In [ ]:
df['marker'] = df['marker'].astype(float)
df['output'] = df['output'].astype(float)

### Population stats
stat_list = [np.mean, np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df.groupby(by=['condition','construct','construct2','biorep','exp'])
stats2 = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats2.columns = stats2.columns.map(lambda i: base.rename_multilevel_cols(i))
stats2['count'] = grouped['marker'].count().reset_index()['marker']
stats2 = stats2.merge(metadata, how='left', on='construct')
stats2.sort_values(['design','ts_kind','ts_num'], inplace=True)
stats2 = stats2.merge(metadata_construct2, how='left', on='construct2')
stats2['condition'] = stats2['construct'] + '_' + stats2['construct2']
display(stats2)

stat_name = {'output_gmean': 'Geometric\nmean', 'output_std': 'Standard\ndeviation', 'output_variation': 'Coefficient\nof variation'}


### Bin by marker quantiles
by = ['construct2','construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles2 = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])

# Quantile stats & slope
df_quantiles2['bin_marker_quantiles_median_log'] = df_quantiles2['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles2['output_log'] = df_quantiles2['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles2.groupby(by=['construct','construct2','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles2 = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles2.columns = stats_quantiles2.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles2['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles2 = stats_quantiles2.merge(metadata, how='left', on='construct')
stats_quantiles2['bin_marker_quantiles_median_log'] = stats_quantiles2['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles2['output_gmean_log'] = stats_quantiles2['output_gmean'].apply(np.log10)


# Compute slope for all constructs
fits2 = stats_quantiles2.groupby(['construct','construct2','exp','biorep'])[stats_quantiles2.columns].apply(base.get_slope).reset_index()
fits2['intercept'] = fits2['intercept_log'].apply(lambda x: 10**x)
fits2 = fits2.merge(metadata, how='left', on='construct')
fits2.sort_values(['design','ts_kind','ts_num'], inplace=True)
fits2 = fits2.merge(metadata_construct2, how='left', on='construct2')
fits2['condition'] = fits2['construct'] + '_' + fits2['construct2']
display(fits2)

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 'ytick.major.size': 3, 'xtick.major.size': 3})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,8))
fig_gridspec = matplotlib.gridspec.GridSpec(4, 6, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[2,2,1.5,2], width_ratios=[1]*6)
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:]),
    'B': fig.add_subfigure(fig_gridspec[1,:2]),
    'C': fig.add_subfigure(fig_gridspec[1,2:4]),
    'D': fig.add_subfigure(fig_gridspec[1,4:]),
    'E': fig.add_subfigure(fig_gridspec[2,:2]),
    'F': fig.add_subfigure(fig_gridspec[2,2:]),
    'G': fig.add_subfigure(fig_gridspec[3,:3]),
    'H': fig.add_subfigure(fig_gridspec[3,3:]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

fig_path = output_path/'fig_architecture.pdf'
fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['A']
rd.plot.adjust_subplot_margins_inches(subfig, left=2.7, bottom=0.45, top=0.35, right=0.1)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.15))

# two-gene architectures with 3'UTR target sites
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = stats2[((stats2['gene']=='1T') & (stats2['design']==1) & (stats2['group']=='controller')) |
                ((stats2['gene']=='2T') & (stats2['group']=='dual') & (stats2['ts_loc']=='3\'')) |
                ((stats2['gene']=='2V') & (stats2['group']=='ts3') & (stats2['construct2_promoter']=='EF1a')) |
                ((stats2['group']=='base') & (stats2['construct2_promoter']!='U6'))].copy()
plot_df.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)
plot_df2 = fits2[((fits2['gene']=='1T') & (fits2['design']==1) & (fits2['group']=='controller')) |
                ((fits2['gene']=='2T') & (fits2['group']=='dual') & (fits2['ts_loc']=='3\'')) |
                ((fits2['gene']=='2V') & (fits2['group']=='ts3') & (fits2['construct2_promoter']=='EF1a')) |
                ((fits2['group']=='base') & (fits2['construct2_promoter']!='U6'))].copy()
plot_df2.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='output_gmean', hue='condition', palette=two_gene_palette,
                    legend=False, ax=ax, marker=two_gene_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_label'], annotate=False, ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='output_std', hue='condition', palette=two_gene_palette,
                    legend=False, ax=ax, marker=two_gene_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1))
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_label'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='slope', hue='condition', palette=two_gene_palette,
                    legend=False, ax=ax, marker=two_gene_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0','','','','1'])
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_label'], annotate=False, ax=ax)
sns.despine(ax=ax)

for i,ax in enumerate(axes):
    ax.axvspan(2.5, 4.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.annotate('1T', (1,ax.get_ylim()[1]), ha='center', va='top', ma='center', fontsize=smaller_size)
    ax.annotate('2T', (3.5,ax.get_ylim()[1]), ha='center', va='top', ma='center', fontsize=smaller_size)
    ax.annotate('2V', (6,ax.get_ylim()[1]), ha='center', va='top', ma='center', fontsize=smaller_size)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.minorticks_off()

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.7, bottom=0.52, top=0.47, right=0.2)
ax = subfig.subplots(1,1,)

xlim = (1e1,1e5)
biorep = 2
plot_df = df_quantiles2[(((df_quantiles2['gene']=='1T') & (df_quantiles2['design']==1) & (df_quantiles2['group']=='controller')) |
                        ((df_quantiles2['gene']=='2T') & (df_quantiles2['group']=='dual') & (df_quantiles2['ts_loc']=='3\'')) |
                        ((df_quantiles2['gene']=='2V') & (df_quantiles2['group']=='ts3') & (df_quantiles2['construct2_promoter']=='EF1a')) |
                        ((df_quantiles2['group']=='base') & (df_quantiles2['construct2_promoter']!='U6'))) & (df_quantiles2['biorep']==biorep)].copy()
plot_df.sort_values(['gene','construct2_promoter'], inplace=True)
sns.kdeplot(data=plot_df, x='output',
                hue='condition', palette=two_gene_palette, ax=ax,
                cumulative=True, common_norm=False, legend=False, log_scale=True)
sns.kdeplot(data=df_quantiles2[df_quantiles2['group']=='marker'], x='output', hue='condition', palette=two_gene_palette,
            cumulative=True, common_norm=False, legend=False, log_scale=True, ax=ax, ls=':', zorder=1)
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both')
sns.despine(ax=ax)
ax.set(xlim=xlim, ylabel='fraction of cells\nbelow given output', title='Cumulative distribution')
ax.minorticks_off()

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.27, bottom=0.52, top=0.47, right=0.15)
ax = subfig.subplots(1,1,)

# width of output range 5-95%
ax.set(ylim=(1e1,1e5), yscale='log', ylabel='', xlim=(-0.5,7.5), title='Output range of\n5-95th percentile')
ax.set_xticklabels(['base','circuit','unreg.']+['']*5, rotation=45, ha='right')
sns.despine(ax=ax)
ax.minorticks_off()

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.27, bottom=0.52, top=0.47, right=0.15)
ax = subfig.subplots(1,1,)

# % within 10x of median
ax.set(ylim=(0,1), ylabel='', xlim=(-0.5,7.5), title='Fraction within\n1 order of magnitude')
ax.set_xticklabels(['base','circuit','unreg.']+['']*5, rotation=45, ha='right')
sns.despine(ax=ax)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.25, bottom=0.5, top=0.35, right=0.1)
axes = subfig.subplots(1,2,)

for ax in axes:
    ax.set(xlim=(1e1,1e5), xscale='log', xlabel='output', title='MOI = ')
    sns.despine(ax=ax)
    ax.minorticks_off()
    ax.get_yaxis().set_visible(False)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.9, bottom=0.4, top=0.4, right=0)
axes = subfig.subplots(1,2, sharey=True, gridspec_kw=dict(width_ratios=(1,0.3)))

xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['miR']=='miRE.FF4') & (df_quantiles['group']=='controller') & (df_quantiles['design']>1) & (df_quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (df_quantiles['promoter']=='EF1a') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=designs_palette, 
             legend=False, dashes=False, style='construct', markers=designs_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker')
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=smaller_size)
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=designs_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
#sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=main_palette, 
#             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='',)
ax.get_xaxis().set_visible(False)
ax.minorticks_off()

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.4, top=0.35, right=0.1)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))

ylim = (2e1,1e5)

# stat gmean
ax = axes[0]
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (stats['promoter']=='EF1a')]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['design'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['design'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1))
ax.minorticks_off()

# slope
ax = axes[2]
plot_df2 = fits[(((fits['miR']=='miRE.FF4') & (fits['group']=='controller') & (fits['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (fits['promoter']=='EF1a')]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['design'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0','','','','1'])
sns.despine(ax=ax)

fig.savefig(rd.outfile(fig_path))